# Processamento do Anexo C

In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
import re
from src.anexo_c.process_dashboard import process_dashboard
from src.anexo_c.export_pdf import export_pdf
from tqdm.notebook import tqdm

## Configurações

In [2]:
load_dotenv()
processamentos = process_dashboard('PIS')

INPUT_DIR = os.getenv('input_dir')
OUTPUT_DIR = os.getenv('output_dir')
INVALID_DIR = os.getenv('invalid_dir')

DASHBOARD_CSV = os.path.join(INPUT_DIR, "Anexo C", "Diligencia_Contratos.csv")
TEMPLATE_PATH = os.path.join(INPUT_DIR, "Anexo C", "template.xlsx")

VALUE_COLS = ['ValorDebito', 'ValorCredito', 'Movimentacao']
GROUP_COLS = ['Conta_Nome','Cosif_Nome']

CONFIGS = [
    (['Total das Receitas', ' Exclusão'], '(01) Total das Receitas'),
    ([' Exclusão'], '(02) Exclusão'),
    ([' Dedução'], '(03) Dedução'),
    (['Sem efeito'],'Sem efeito no Pis Cofins'),
]

## Geração do Excel

In [3]:
dashboard = pd.read_csv(DASHBOARD_CSV, sep=';', encoding='latin1')

In [4]:
contratos = dashboard['NumContrato'].unique().tolist()
contratos_selecionados = pd.read_csv(r"100 Kits_KPMG.csv")['Contrato'].tolist()
contratos_to_process = [c for c in contratos if c in contratos_selecionados]
contratos_to_process = contratos_to_process[:1]

In [5]:
processamentos.get_contract_numbers(OUTPUT_DIR)

✅ Extraídos 1 itens e salvos em 'numeros_extraidos.txt'


In [6]:
nao_encontrados_3 = set()
if os.path.isfile("numeros_extraidos.txt"):
    with open("numeros_extraidos.txt", "r") as f:
        nao_encontrados_3 = set(line.strip() for line in f)

# Filtre os contratos removendo aqueles que estão na lista de não encontrados
contratos = [contrato for contrato in contratos_to_process if str(contrato) not in nao_encontrados_3]

for contrato in tqdm(contratos, unit="contrato"):
    tqdm.desc = f"processando contrato: {contrato}"
    dashboard_filtro = dashboard[(dashboard['NumContrato'] == contrato)]


    dashboard_filtro = processamentos.replicate_years(dashboard_filtro,
                                                                group_cols=GROUP_COLS,
                                                                value_cols=VALUE_COLS,
                                                                fill_value=0)
    final = pd.DataFrame()

# para cada filtro, gera o bloco e concatena
    for filtro, desc in CONFIGS:
        bloco = processamentos.group_revenues(dashboard_filtro, filtro, desc)
        final = pd.concat([final, bloco], ignore_index=True)



    df_pis_cofins = processamentos.calcula_pis_cofins(final)



    final = pd.concat([final, df_pis_cofins], ignore_index=True)


    final = final[['Ano', 'Descrição', 'Conta_Nome', 'Cosif_Nome', 'ValorCredito', 'Movimentacao']]

    df = final.copy()
    output_path   = os.path.join(OUTPUT_DIR, f"C{contrato}.xlsx")

    processamentos.atualizar_template_pivot(
        template_path=TEMPLATE_PATH,
        output_path=output_path,
        df=df,
        contrato=contrato
    )


0contrato [00:00, ?contrato/s]

## Exportação do PDF

In [7]:
export_pdf().process_file()

Verificando arquivos inválidos...


Verificando: 100%|██████████| 2/2 [00:00<00:00, 24.77it/s]


Renomeando arquivos...
Convertendo para PDF...


Convertendo: 100%|██████████| 1/1 [00:26<00:00, 26.03s/it]

✅ Processo finalizado.


# ( •̀ ω •́ )y